<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 22s 828ms/step - loss: 0.6735 - accuracy: 0.6083 - val_loss: 0.6727 - val_accuracy: 0.5122
Epoch 2/10
26/26 [==============================] - 22s 838ms/step - loss: 0.6112 - accuracy: 0.6849 - val_loss: 0.5801 - val_accuracy: 0.8098
Epoch 3/10
26/26 [==============================] - 22s 850ms/step - loss: 0.5515 - accuracy: 0.7384 - val_loss: 0.7252 - val_accuracy: 0.5268
Epoch 4/10
26/26 [==============================] - 22s 842ms/step - loss: 0.4920 - accuracy: 0.7762 - val_loss: 0.5820 - val_accuracy: 0.6537
Epoch 5/10
26/26 [==============================] - 23s 869ms/step - loss: 0.4258 - accuracy: 0.8200 - val_loss: 0.4410 - val_accuracy: 0.8195
Epoch 6/10
26/26 [==============================] - 24s 941ms/step - loss: 0.3826 - accuracy: 0.8418 - val_loss: 0.4280 - val_accuracy: 0.8244
Epoch 7/10
26/26 [==============================] - 22s 852ms/step - loss: 0.3420 - accuracy: 0.8601 - val_loss: 1.4149 - val_accuracy: 0.4585

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.68.h5
26/26 - 27s - loss: 0.6755 - accuracy: 0.5560 - val_loss: 0.6770 - val_accuracy: 0.4780
Epoch 2/5

Epoch 00002: saving model to weights.02-0.61.h5
26/26 - 23s - loss: 0.6254 - accuracy: 0.6825 - val_loss: 0.6073 - val_accuracy: 0.6390
Epoch 3/5

Epoch 00003: saving model to weights.03-0.56.h5
26/26 - 23s - loss: 0.5995 - accuracy: 0.6800 - val_loss: 0.5628 - val_accuracy: 0.6683
Epoch 4/5

Epoch 00004: saving model to weights.04-0.57.h5
26/26 - 23s - loss: 0.5535 - accuracy: 0.7190 - val_loss: 0.5676 - val_accuracy: 0.6634
Epoch 5/5

Epoch 00005: saving model to weights.05-0.61.h5
26/26 - 22s - loss: 0.4818 - accuracy: 0.8029 - val_loss: 0.6056 - val_accuracy: 0.6000


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 27s - loss: 0.6780 - accuracy: 0.5584 - val_loss: 0.6750 - val_accuracy: 0.5268


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 24s - loss: 0.6774 - accuracy: 0.5608 - val_loss: 0.6540 - val_accuracy: 0.7366
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 21s - loss: 0.6185 - accuracy: 0.6983 - val_loss: 0.5978 - val_accuracy: 0.6780


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 23s - loss: 0.6867 - accuracy: 0.5791 - val_loss: 0.6657 - val_accuracy: 0.6439
Epoch 2/50
26/26 - 20s - loss: 0.6388 - accuracy: 0.6642 - val_loss: 0.6193 - val_accuracy: 0.7561
Epoch 3/50
26/26 - 21s - loss: 0.5933 - accuracy: 0.7336 - val_loss: 0.5751 - val_accuracy: 0.7707
Epoch 4/50
26/26 - 20s - loss: 0.5631 - accuracy: 0.7457 - val_loss: 0.5283 - val_accuracy: 0.8049
Epoch 5/50
26/26 - 20s - loss: 0.5497 - accuracy: 0.7494 - val_loss: 0.4752 - val_accuracy: 0.8293
Epoch 6/50
26/26 - 20s - loss: 0.4676 - accuracy: 0.7956 - val_loss: 0.4215 - val_accuracy: 0.8537
Epoch 7/50
26/26 - 20s - loss: 0.3991 - accuracy: 0.8345 - val_loss: 0.3438 - val_accuracy: 0.8878
Epoch 8/50
26/26 - 20s - loss: 0.3667 - accuracy: 0.8686 - val_loss: 0.3360 - val_accuracy: 0.8976
Epoch 9/50
26/26 - 20s - loss: 0.3144 - accuracy: 0.8856 - val_loss: 0.2954 - val_accuracy: 0.9073
Epoch 10/50
26/26 - 21s - loss: 0.2538 - accuracy: 0.9124 - val_loss: 0.2851 - val_accuracy: 0.8976
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 21s 791ms/step - loss: 0.6706 - accuracy: 0.5657 - val_loss: 0.6563 - val_accuracy: 0.6732
Epoch 2/5
26/26 [==============================] - 20s 781ms/step - loss: 0.6379 - accuracy: 0.6727 - val_loss: 0.6119 - val_accuracy: 0.6341
Epoch 3/5
26/26 [==============================] - 20s 788ms/step - loss: 0.5868 - accuracy: 0.7348 - val_loss: 0.6196 - val_accuracy: 0.5512
Epoch 4/5
26/26 [==============================] - 21s 789ms/step - loss: 0.5313 - accuracy: 0.7749 - val_loss: 0.4728 - val_accuracy: 0.8878
Epoch 5/5
26/26 [==============================] - 21s 792ms/step - loss: 0.4602 - accuracy: 0.8358 - val_loss: 0.4412 - val_accuracy: 0.8390


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.565693,0.670782,0.673171,0.656276
1,1,0.672749,0.638241,0.634146,0.611943
2,2,0.734793,0.587295,0.551219,0.619625
3,3,0.774939,0.531838,0.887805,0.472776
4,4,0.835766,0.460760,0.839024,0.441206


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 31s 1s/step - loss: 0.6732 - accuracy: 0.5584 - val_loss: 0.6658 - val_accuracy: 0.8049

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 21s 795ms/step - loss: 0.6550 - accuracy: 0.6350 - val_loss: 0.6561 - val_accuracy: 0.7220

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 773ms/step - loss: 0.6437 - accuracy: 0.7007 - val_loss: 0.6569 - val_accuracy: 0.5707

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
24/26 [==========================>...] - ETA: 1s - loss: 0.6388 - accuracy: 0.6706

In [ ]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

In [ ]:
%tensorboard --logdir log_dir